# API Center ❤️ all APIs

## Import from Kong API Gateway
![image](../../images/import-from-kong.png)

Playground to experiment importing APIs from Kong API Gateway. We start by creating an API Center instance using [Bicep resource definition](https://learn.microsoft.com/en-us/azure/templates/microsoft.apicenter/services?pivots=deployment-language-bicep) and then we will use the [Kong Admin API](https://docs.konghq.com/gateway/api/admin-oss/latest) to export Kong services and import them into API Center.

💡 Every step outlined below leverages the power of PowerShell scripts. This is designed so you can seamlessly integrate them into your automation workflows, CI/CD pipelines, webhooks, and more.

### Learning Objectives - Upon completing this lab, you should be able to:
- Deploy Azure API Center using Bicep.
- Configure the API Center metadata scheme for APIs, Environments and Deployments.
- Understand the Azure CLI commands to manage Azure API Center. [Full list of commands available here](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest).
- Create an enviroment with custom metadata properties 
- Register an API using the OpenAPI spec as the source of truth and assign API metadata properties.
- Register an API with individual commands to have full control over all the properties. 
- Search and discover the APIs registered in API Center.

### Prerequisites
- Install or update to the latest [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download)
- Install or update to the latest [Visual Studio Code](https://code.visualstudio.com/)
- Install the [Polyglot Notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) from the VS Code marketplace
- Install or update to the latest [PowerShell](https://learn.microsoft.com/en-us/powershell/scripting/install/installing-powershell)
- Install or update to the latest [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli). The Azure API Center extension will automatically install the first time you run an [az apic](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) command.
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)


### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the APIC location parameter according your preferences and [region availability.](https://learn.microsoft.com/en-us/azure/api-center/overview#available-regions) 


In [ ]:

$kongAdminAPIUrl = "http://localhost:8001"

$deploymentName = Split-Path -Path (Get-Location) -Leaf
$resourceGroupName = "lab-$deploymentName" # change the name to match your naming style
$resourceGroupLocation = "westeurope"
$apicResourceNamePrefix = "apic"
$apicResourceSku = "free"
$apicResourceTags = @{
  "lab" = $deploymentName
}

# the following metadata will be created during the deployment
$metadata = @(
  @{
    "name" = 'kongVersion'
    "schema" = '{"title":"kongVersion","description":"The Kong API Gateway version","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'kongEdition'
    "schema" = '{"title":"kongEdition","description":"The Kong API Gateway edition","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'konnectMode'
    "schema" = '{"title":"konnectMode","description":"Using Kong Konnectn","type":"boolean"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'upstreamHost'
    "schema" = '{"title":"host","description":"Upstream server","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'apiEnabled'
    "schema" = '{"title":"apiEnabled","description":"The API is enabled","type":"boolean"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'tags'
    "schema" = '{"title":"tags","description":"API tags","type":"array","items":{"type":"string"}}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'protocol'
    "schema" = '{"title":"protocol","description":"The API deployment protocol","type":"boolean"}'
    "assignedTo" = @(
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  } 
)


# the following environment(s) will be created during the deployment
$environments = @(
  @{
    "title" = 'dev'
    "kind" = 'development'
    "description" = 'Dev Environment'
    "server" = @{
        "type" = 'Kong API Gateway' 
        "managementPortalUri" = 'https://localhost:8002/'
    }
    "onboarding" = @{
        "developerPortalUri" = ''
        "instructions" = ''
    }
    "customProperties" = @{ 
    }
  }
)



Write-Output "✅ Variables initialized ⌚ $(Get-Date -Format 'HH:mm:ss')"


### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
$resourceGroupOutput = az group create --name $resourceGroupName --location $resourceGroupLocation

if ($LASTEXITCODE -ne 0) {
    Write-Output $resourceGroupOutput
} else {
    Write-Output "✅ Azure Resource Grpup $resourceGroupName created ⌚ $(Get-Date -Format 'HH:mm:ss')"
}

### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
$bicepParameters = @{
    "`$schema" = "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#"
    "contentVersion" = "1.0.0.0"
    "parameters" = @{
        "apicResourceNamePrefix" = @{ "value" = $apicResourceNamePrefix }
        "apicResourceTags" = @{ "value" = $apicResourceTags }
        "apicMetadataSchema" = @{ "value" = $metadata }
        "apicEnvironments" = @{ "value" = $environments }
    }
}

$bicepParametersJson = ConvertTo-Json -InputObject $bicepParameters -Depth 10
Set-Content -Path "params.json" -Value $bicepParametersJson

# Execute the Azure CLI command to create the deployment
az deployment group create --name $deploymentName --resource-group $resourceGroupName --template-file "main.bicep" --parameters "params.json"



### 3️⃣ Get the deployment outputs

We will set the `apicResourceName` variable with the value that was returned from the deployment 

In [ ]:
$deploymentOutput = az deployment group show --name $deploymentName -g $resourceGroupName --query properties.outputs.apicResourceName.value -o json
if ($LASTEXITCODE -ne 0) {
    Write-Output $deploymentOutput
} else {
    $apicResourceName = $deploymentOutput | ConvertFrom-Json
}
Write-Output "👉🏻 API Center name: $apicResourceName"

### 4️⃣ Update the environment metadata

We will [update the enviroment](https://learn.microsoft.com/en-us/cli/azure/apic/environment?view=azure-cli-latest#az-apic-environment-update) with the Kong configuration

In [ ]:
$config = Invoke-RestMethod -Uri $kongAdminAPIUrl/ -Method Get
$environmentCustomProperties = @{
    "kongVersion" = $config.version
    "kongEdition" = $config.edition
    "konnectMode" = $config.configuration.konnect_mode
}

$environmentCustomPropertiesJson = ConvertTo-Json -InputObject $environmentCustomProperties -Depth 10 -Compress

az apic environment update -g $resourceGroupName -s $apicResourceName --environment-id "dev" `
            --custom-properties $environmentCustomPropertiesJson.replace('"','\"')

### 5️⃣ Retrieves the Kong services and import into API Center

It uses the [Kong Admin API](https://docs.konghq.com/gateway/latest/get-started/services-and-routes/) to retrieve the services and then the Azure CLI to import the APIs into API Center.

In [ ]:

$services = Invoke-RestMethod -Uri $kongAdminAPIUrl/services -Method Get

foreach ($service in $services.data) {
    $apiCustomProperties = @{
        "upstreamHost" = $service.host
        "apiEnabled" = $service.enabled
        "tags" = $service.tags
    }

    $apiId = $service.name
    $apiName = $service.name
    $apiVersionId = "1-0"
    $apiVersionName = "1.0"
    $definitionId = "openapi"

    $apiCustomPropertiesJson = ConvertTo-Json -InputObject $apiCustomProperties -Depth 10 -Compress
    write-output "👉🏻 Creating API $apiName"
    az apic api create -g $resourceGroupName -s $apicResourceName --api-id $apiId --type REST `
        --title $apiName  `
        --custom-properties $apiCustomPropertiesJson.replace('"','\"')

    write-output "👉🏻 Creating version $apiVersion of API $apiName"
    az apic api version create -g $resourceGroupName -s $apicResourceName --api-id $apiId --version-id $apiVersionId  `
                --title $apiVersionName --lifecycle-stage "development"

    write-output "👉🏻 Creating definition for version $apiVersion of API $apiName"
    az apic api definition create -g $resourceGroupName -s $apicResourceName --api-id $apiId `
        --version-id $apiVersionId --definition-id $definitionId --title "OpenAPI" --description "OpenAPI spec"      

    $deploymentCustomProperties = @{
        "protocol" = $service.protocol
    }
    
    $deploymentCustomPropertiesJson = ConvertTo-Json -InputObject $deploymentCustomProperties -Depth 10 -Compress
    write-output "👉🏻 Creating deployment for the API $apiName"
    az apic api deployment create -g $resourceGroupName -s $apicResourceName --deployment-id "deployment" `
            --title "Development deployment" --description "Development deployment." --api-id $apiId `
            --environment-id "/workspaces/default/environments/dev" `
            --definition-id "/workspaces/default/apis/another-petstore/versions/$apiVersionId/definitions/$definitionId" `
            --server '{\"runtimeUri\":[\"http://localhost:8000\"]}' `
            --custom-properties $deploymentCustomPropertiesJson.replace('"','\"')


}

### 6️⃣ Discover the APIs that were just imported

You can discover the APIs with fhe following methods:
- With the Azure Portal
- With the [self-hosted API Center Portal](https://learn.microsoft.com/en-us/azure/api-center/enable-api-center-portal)
- With the [VS Code extension](https://learn.microsoft.com/en-us/azure/api-center/use-vscode-extension-copilot) that is integrated with GitHub Copilot Chat.
- With the CLI, the service REST API and more

Here we will use the [list command](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-list) to display the APIs that we have just imported. 


In [ ]:
az apic api list -g $resourceGroupName -s $apicResourceName --query "[].{Name:name, Title:title, Kind:kind, ContactEmail:contacts[0].email}" -o table

### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.